# I Ging Oracle Testing & Development Notebook

Dieses Notebook dient zum Testen und Erweitern des I Ging Orakels. Es demonstriert:
1. Grundlegende Oracle-Funktionen
2. Erweiterte Weissagungen mit vollständigen Texten
3. Verschiedene Konsultationsmodi
4. Visualisierung der Hexagramme

## Setup
Zuerst importieren wir alle benötigten Module und initialisieren das Logging.

In [ ]:
import os
import logging
from pathlib import Path
from typing import List, Dict, Any
from IPython.display import display, Markdown, HTML

from yijing.oracle import (YijingOracle,
                     ask_oracle,
                     generiere_erweiterte_weissagung,
                     formatiere_weissagung_markdown,
                     analysiere_hexagramm_eigenschaften,
                     formatiere_analyse_markdown
                    )


from yijing.models import Hypergram, HypergramLine, HypergramData
from yijing.managers import HexagramManager
from yijing.enums import ConsultationMode

from jupyter_json_viewer import display_json

# Logging Setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Verzeichnisse initialisieren
project_dir = Path.cwd()
resources_dir = project_dir / 'yijing' / 'resources'

# Oracle initialisieren
oracle = YijingOracle(
    api_key=os.environ["API_KEY"],
    custom_settings={
        "model_name": "models/gemini-1.5-pro-latest"
        #"model_name": "models/gemini-1.5-flash-latest"
    }
)

## Erweiterte Weissagungsfunktion

Diese Version der Weissagungsfunktion enthält alle relevanten Texte aus den JSON-Files und formatiert sie übersichtlich.

In [9]:
import json

# load all json files from resources directory / hexagram_json
hexagram_json_files = [f for f in resources_dir.glob('hexagram_json/*.json') if f.is_file()]

# convert to json objects
hexagram_jsons = []
for file in hexagram_json_files:
    with open(file, 'r') as f:
        hexagram_jsons.append(json.load(f))

In [ ]:
display_json(hexagram_jsons[0], collapsed=True, dark_mode=True)

## Weissagung durchführen

Jetzt können wir eine vollständige Weissagung durchführen und die Ergebnisse anzeigen.

In [ ]:
from yijing.oracle import generiere_erweiterte_weissagung, formatiere_weissagung_markdown

In [ ]:
# Beispiel mit spezifischen Linienwerten
linien_werte = [8, 7, 6, 9, 9, 6]  # Beispielwerte
weissagung = generiere_erweiterte_weissagung(linien_werte)
markdown_text = formatiere_weissagung_markdown(weissagung)
display(Markdown(markdown_text))

## Oracle Konsultation

Nun können wir das Oracle zu einer spezifischen Frage konsultieren.

In [ ]:
# Frage an das Oracle
frage = "Wie wird die Veranstaltung?"
antwort = oracle.get_response(frage)

print("FRAGE:")
print(frage)
print("\nANTWORT DES ORAKELS:")
display(Markdown(antwort['answer']))

## Dialog-Modus Test

Hier testen wir den Dialog-Modus des Orakels für eine interaktivere Konsultation.

In [ ]:
# Oracle im Dialog-Modus initialisieren
dialog_oracle = YijingOracle(
    api_key=os.environ["API_KEY"],
    custom_settings={
        "model_name": "models/gemini-1.5-pro-latest",
        "consultation_mode": ConsultationMode.DIALOGUE
    }
)

# Neue Konsultation starten
dialog_oracle.start_new_consultation()

# Erste Frage stellen
frage = "Wie kann ich innere Balance finden?"
antwort = dialog_oracle.get_response(frage)

print("FRAGE 1:")
print(frage)
print("\nANTWORT:")
display(Markdown(antwort['answer']))

# Folge-Frage stellen
frage = "Wie kann ich die genannten Ratschläge am besten in meinen Alltag integrieren?"
antwort = dialog_oracle.get_response(frage)

print("\nFRAGE 2:")
print(frage)
print("ANTWORT:")
display(Markdown(antwort['answer']))

## Erweiterte Funktionen

In [ ]:
'''
def analysiere_hexagramm_eigenschaften(weissagung: Dict[str, Any]) -> Dict[str, Any]:
    """Analysiert die Eigenschaften und Beziehungen der Hexagramme in einer Weissagung."""
    ursprung = weissagung['ursprung']
    ergebnis = weissagung['ergebnis']
    
    return {
        'trigramm_transformation': {
            'ursprung': {
                'oben': ursprung['trigrams']['above'],
                'unten': ursprung['trigrams']['below']
            },
            'ergebnis': {
                'oben': ergebnis['trigrams']['above'],
                'unten': ergebnis['trigrams']['below']
            }
        },
        'wandlungslinien_anzahl': len(weissagung['wandelnde_linien']['positionen']),
        'wandlungslinien_positionen': weissagung['wandelnde_linien']['positionen'],
        'kernaspekte': {
            'ursprung': {
                'name': ursprung['name'],
                'kernelement': ursprung['bedeutung']['description'].split('.')[0]
            },
            'ergebnis': {
                'name': ergebnis['name'],
                'kernelement': ergebnis['bedeutung']['description'].split('.')[0]
            }
        }
    }

# Funktion zum Formatieren der Analyse
def formatiere_analyse_markdown(analyse: Dict[str, Any]) -> str:
    """Formatiert die Hexagramm-Analyse als Markdown."""
    md = [
        "# ANALYSE DER HEXAGRAMM-TRANSFORMATION\n",
        
        "## Trigramm-Transformation\n",
        "### Ursprüngliches Hexagramm\n",
        f"- Oberes Trigramm: {analyse['trigramm_transformation']['ursprung']['oben']['name']} "
        f"({analyse['trigramm_transformation']['ursprung']['oben']['attributes']})\n",
        f"- Unteres Trigramm: {analyse['trigramm_transformation']['ursprung']['unten']['name']} "
        f"({analyse['trigramm_transformation']['ursprung']['unten']['attributes']})\n",
        
        "### Resultierendes Hexagramm\n",
        f"- Oberes Trigramm: {analyse['trigramm_transformation']['ergebnis']['oben']['name']} "
        f"({analyse['trigramm_transformation']['ergebnis']['oben']['attributes']})\n",
        f"- Unteres Trigramm: {analyse['trigramm_transformation']['ergebnis']['unten']['name']} "
        f"({analyse['trigramm_transformation']['ergebnis']['unten']['attributes']})\n",
        
        "## Wandlungslinien\n",
        f"- Anzahl der Wandlungen: {analyse['wandlungslinien_anzahl']}\n",
        f"- Positionen: {', '.join(map(str, analyse['wandlungslinien_positionen']))}\n",
        
        "## Kernaspekte\n",
        "### Ursprüngliches Hexagramm\n",
        f"- Name: {analyse['kernaspekte']['ursprung']['name']}\n",
        f"- Kernelement: {analyse['kernaspekte']['ursprung']['kernelement']}\n",
        
        "### Resultierendes Hexagramm\n",
        f"- Name: {analyse['kernaspekte']['ergebnis']['name']}\n",
        f"- Kernelement: {analyse['kernaspekte']['ergebnis']['kernelement']}\n"
    ]
    
    return "".join(md)
'''

### 1. Gesamtanalyse einer Weissagung
Hier führen wir eine vollständige Analyse durch, die sowohl die Weissagung als auch die statistischen Eigenschaften umfasst.

In [ ]:
# Vollständige Analyse durchführen
weissagung = generiere_erweiterte_weissagung(linien_werte)
analyse = analysiere_hexagramm_eigenschaften(weissagung)

# Weissagung und Analyse anzeigen
print("=== VOLLSTÄNDIGE I GING ANALYSE ===")
display(Markdown(formatiere_weissagung_markdown(weissagung)))
print("\n=== STATISTISCHE ANALYSE ===")
display(Markdown(formatiere_analyse_markdown(analyse)))

## Experimentelle Features

### 1. Vergleich mehrerer Weissagungen
Diese Funktion ermöglicht es, Muster in mehreren aufeinanderfolgenden Weissagungen zu erkennen.

In [ ]:

import random
import json

def vergleiche_weissagungen(anzahl: int = 3) -> Dict[str, Any]:
    """Führt mehrere Weissagungen durch und vergleicht ihre Eigenschaften."""
    weissagungen = []
    for _ in range(anzahl):
        # Zufällige Linienwerte generieren
        linien = [random.choice([6, 7, 8, 9]) for _ in range(6)]
        weissagung = generiere_erweiterte_weissagung(linien)
        weissagungen.append(weissagung)
    
    # Muster analysieren
    muster = {
        'häufige_hexagramme': {},
        'wandlungsstatistik': {
            'durchschnitt_wandlungen': 0,
            'häufige_positionen': {}
        }
    }
    
    # Hexagramme zählen
    for w in weissagungen:
        # Ursprungshexagramm
        ursprung = w['ursprung']['name']
        muster['häufige_hexagramme'][ursprung] = \
            muster['häufige_hexagramme'].get(ursprung, 0) + 1
        
        # Ergebnishexagramm
        ergebnis = w['ergebnis']['name']
        muster['häufige_hexagramme'][ergebnis] = \
            muster['häufige_hexagramme'].get(ergebnis, 0) + 1
        
        # Wandlungsstatistik
        wandlungen = len(w['wandelnde_linien']['positionen'])
        muster['wandlungsstatistik']['durchschnitt_wandlungen'] += wandlungen
        
        for pos in w['wandelnde_linien']['positionen']:
            muster['wandlungsstatistik']['häufige_positionen'][pos] = \
                muster['wandlungsstatistik']['häufige_positionen'].get(pos, 0) + 1
    
    # Durchschnitt berechnen
    muster['wandlungsstatistik']['durchschnitt_wandlungen'] /= anzahl
    
    return muster

# Vergleich durchführen
vergleichsergebnisse = vergleiche_weissagungen(5)
print("Vergleichsanalyse von 5 Weissagungen:")
print(json.dumps(vergleichsergebnisse, indent=2, ensure_ascii=False))

### 2. Interaktive Weissagungsserie
Diese Funktion ermöglicht es, eine Serie von Weissagungen zu einer zusammenhängenden Fragestellung durchzuführen.

Führt eine Serie von zusammenhängenden Weissagungen durch.


In [5]:
hauptfrage = "Werde ich es heute schaffen?"

In [ ]:
print(f"=== WEISSAGUNGSSERIE ZU: {hauptfrage} ===\n")

# Oracle im Dialogmodus initialisieren
series_oracle = YijingOracle(
    api_key=os.environ["API_KEY"],
    custom_settings={
        "consultation_mode": ConsultationMode.DIALOGUE
    }
)

# Neue Konsultation starten
series_oracle.start_new_consultation()

# Hauptfrage stellen
print("HAUPTFRAGE:")
antwort = series_oracle.get_response(hauptfrage)
display(Markdown(antwort['answer']))

In [7]:
nachfrage_1 = "Sollte zuerst die alten Sachen abarbeiten?"

In [ ]:
print(f"\nNACHFRAGE 1:")
print(nachfrage_1)
antwort_auf_nachfrage_1 = series_oracle.get_response(nachfrage_1)
display(Markdown(antwort_auf_nachfrage_1['answer']))

In [ ]:

# Unterfragen behandeln
for i, frage in enumerate(unterfragen, 1):
    print(f"\nUNTERFRAGE {i}:")
    print(frage)
    antwort = series_oracle.get_response(frage)
    display(Markdown(antwort['answer']))


In [ ]:

# Beispiel einer Weissagungsserie
hauptfrage = "Auf was muss ich nun achten?"
unterfragen = [
    "Welche konkreten Schritte sollte ich als nächstes unternehmen?",
    "Woher nehme ich die Kraft?",
    "Wird es mir gelingen?"
]

führe_weissagungsserie(hauptfrage, unterfragen)

In [ ]:
def führe_weissagungsserie(hauptfrage: str, unterfragen: List[str]):
    """Führt eine Serie von zusammenhängenden Weissagungen durch."""
    print(f"=== WEISSAGUNGSSERIE ZU: {hauptfrage} ===\n")
    
    # Oracle im Dialogmodus initialisieren
    series_oracle = YijingOracle(
        api_key=os.environ["API_KEY"],
        custom_settings={
            "consultation_mode": ConsultationMode.DIALOGUE
        }
    )
    
    # Neue Konsultation starten
    series_oracle.start_new_consultation()
    
    # Hauptfrage stellen
    print("HAUPTFRAGE:")
    antwort = series_oracle.get_response(hauptfrage)
    display(Markdown(antwort['answer']))
    
    # Unterfragen behandeln
    for i, frage in enumerate(unterfragen, 1):
        print(f"\nUNTERFRAGE {i}:")
        print(frage)
        antwort = series_oracle.get_response(frage)
        display(Markdown(antwort['answer']))

# Beispiel einer Weissagungsserie
hauptfrage = "Auf was muss ich nun achten?"
unterfragen = [
    "Welche konkreten Schritte sollte ich als nächstes unternehmen?",
    "Woher nehme ich die Kraft?",
    "Wird es mir gelingen?"
]

führe_weissagungsserie(hauptfrage, unterfragen)

# X

In [2]:
hauptfrage = """Auf was sollten wir während der Aufstellungsversammlung achten?"""

In [ ]:
"""Führt eine Serie von zusammenhängenden Weissagungen durch."""
print(f"=== WEISSAGUNGSSERIE ZU: {hauptfrage} ===\n")

# Oracle im Dialogmodus initialisieren
series_oracle = YijingOracle(
    api_key=os.environ["API_KEY"],
    custom_settings={
        "consultation_mode": ConsultationMode.DIALOGUE,
        "model_name": "models/gemini-1.5-pro-latest"
    }
)

# Neue Konsultation starten
series_oracle.start_new_consultation()

# Hauptfrage stellen
print("HAUPTFRAGE:")
antwort = series_oracle.get_response(hauptfrage)
display(Markdown(antwort['answer']))


In [32]:
unterfrage_1 = "xxx"

In [ ]:
print(unterfrage_1)
antwort = series_oracle.get_response(unterfrage_1)
display(Markdown(antwort['answer']))

In [ ]:

# Unterfragen behandeln
for i, frage in enumerate(unterfragen, 1):
    print(f"\nUNTERFRAGE {i}:")
    print(frage)
    antwort = series_oracle.get_response(frage)
    display(Markdown(antwort['answer']))

# Beispiel einer Weissagungsserie
hauptfrage = "Auf was muss ich nun achten?"
unterfragen = [
"Welche konkreten Schritte sollte ich als nächstes unternehmen?",
"Woher nehme ich die Kraft?",
"Wird es mir gelingen?"
]

führe_weissagungsserie(hauptfrage, unterfragen)